In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from matplotlib import font_manager, rc
import platform

# seaborn 설정 리셋
sns.reset_defaults()

# 폰트설정
if platform.system() == 'Windows' :
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
else :
    print('Check your OS System')
    
# 그래프에 마이너스 표시
matplotlib.rcParams['axes.unicode_minus'] = False

In [ ]:
# 머신러닝 : 알고리즘 선택 O -> 하이퍼파라미터튜닝 -> 결과값(모델) 평가(Accuracy, r)
# 딥러닝 : 알고리즘 선택 X -> 네트워크 구성(w집합) -> 예측(Predict) -> loss-> loss미분 -> 평가(Accuracy, loss) -> 반복
    # 네트워크구성, loss계산법, loss미분계산법

In [ ]:
# 미분
# x = [1,2,3,4], 네트워크(w)는 a  -> predict a+b, 2a+b, 3a+b, 4a+b -> loss(a+b), loss(2a+b), loss(3a+b), loss(4a+b)

In [23]:
from sklearn.datasets import load_iris

In [24]:
iris = load_iris()

In [315]:
data = iris['data']
target = iris['target']

In [26]:
def sigmoid(x) :
    return 1/(1+np.exp(-x))

In [60]:
def softmax(x) :
    c = np.max(x, axis=1).reshape(-1,1)
    x = x-c
    return np.exp(x)/np.sum(np.exp(x),axis=1).reshape(-1,1)

In [63]:
np.sum(softmax(data))

150.0

In [311]:
def categorical_crossentropy(t,y):
    return np.mean(-t * np.log(y))

In [312]:
def make_one_hot(x):
    result = np.zeros((x.size, np.unique(x).size))
    for idx1, idx2 in enumerate(x):
        result[idx1, idx2] = 1
    return result

In [313]:
data.shape

(150, 4)

In [316]:
target = make_one_hot(target)

In [35]:
input_shape = data.shape[1:]
output_shape = target.shape[1:]

In [342]:
def numerical_gradient(f,x):
    h = 1e-4
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=["multi_index"],op_flags=["readwrite"])
    
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh = f(x)
        x[idx] = tmp_val
        fx = f(x)
        grad[idx] = (fxh-fx)/h
        it.iternext()
    return grad

In [343]:
def gradient_descent(x,t) :
    y = predict(x)
    loss = categorical_crossentropy(y, t)
    w_loss = lambda w: categorical_crossentropy(y,t) 
    dw1 = numerical_gradient(w_loss,w1)
    db1 = numerical_gradient(w_loss, b1)
    dw2 = numerical_gradient(w_loss, w2)
    db2 = numerical_gradient(w_loss, b2)
    w1 -= dw1 * learning_rate
    b1 -= db1 * learning_rate
    w2 -= dw2 * learning_rate
    b2 -= db2 * learning_rate

In [365]:
class Network:
    def __init__(self):
        self.w1 = np.random.randn(4, 50)
        self.b1 = np.zeros(50)
        self.w2 = np.random.randn(50, 3)
        self.b2 = np.zeros(3)
        self.err = None
        
    def predict(self,x):
        y = np.dot(x,self.w1) + self.b1
        y = sigmoid(y)
        y = np.dot(y,self.w2) + self.b2
        y = softmax(y)
        return y

    def loss(self,x,t):
        y = self.predict(x)
        err = categorical_crossentropy(t,y)
        self.err = err
        return err
    
    def gradient(self,x,t):
        # self.loss(x,t)
        learning_rate = 1e-4
        w_loss = lambda w : self.loss(x,t)
        dw1 = numerical_gradient(w_loss,self.w1)
        db1 = numerical_gradient(w_loss,self.b1)
        dw2 = numerical_gradient(w_loss,self.w2)
        db2 = numerical_gradient(w_loss,self.b2)
        
        self.w1 -= dw1*learning_rate
        self.b1 -= db1*learning_rate        
        self.w2 -= dw2*learning_rate        
        self.b2 -= db2*learning_rate  
        
        return self.err

In [366]:
model = Network()

In [367]:
np.argmax(model.predict(data[0:1]))

0

In [368]:
model.gradient(data, target)

1.8425239114172407